In [1]:
from pprint import pprint
import gurobipy as gp
from gurobipy import GRB

# Parameters

In [2]:
arcs, days = gp.multidict({
    ('Terry', 'Customer01'): 10,
    ('Terry', 'Customer02'): 15,
    ('Terry', 'Customer03'):  9,
    #
    ('Carle', 'Customer01'):  9,
    ('Carle', 'Customer02'): 18,
    ('Carle', 'Customer03'):  5,
    #
    ('McClymonds', 'Customer01'):  6,
    ('McClymonds', 'Customer02'): 14,
    ('McClymonds', 'Customer03'):  3,
})

supply = {
    'Terry': 1,
    'Carle':  1,
    'McClymonds': 1,
}

demand = {
    'Customer01': 1,
    'Customer02': 1,
    'Customer03': 1,
}

# Model
## Decision Variables

In [3]:
m = gp.Model('assignment')
assign = m.addVars(arcs, vtype=GRB.CONTINUOUS, name='assign')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-19


## Objective Function

In [4]:
Z = assign.prod(days)
m.ModelSense = GRB.MINIMIZE 
m.setObjective(Z)

## Constraints

In [5]:
source = m.addConstrs(
    (assign.sum(i,'*') <= s for  i,s in supply.items()),
    name='leaders'
)

sink = m.addConstrs(
    (assign.sum('*', j) == d for j,d in demand.items()),
    name='customers'
)

In [6]:
m.update()
print(f'Number of constraints {m.NumConstrs}')
print(f'Number of variables {m.NumVars}')
print('Constraints are')
pprint(m.getConstrs())

Number of constraints 6
Number of variables 9
Constraints are
[<gurobi.Constr leaders[Terry,1]>,
 <gurobi.Constr leaders[Carle,1]>,
 <gurobi.Constr leaders[McClymonds,1]>,
 <gurobi.Constr customers[Customer01,1]>,
 <gurobi.Constr customers[Customer02,1]>,
 <gurobi.Constr customers[Customer03,1]>]


## Optimization process

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x6cc8bf17
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3000000e+01   2.000000e+00   0.000000e+00      0s
       2    2.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.600000000e+01


## Results

In [8]:
print(f'Objective Value {m.objVal} days')

for v in m.getVars():
    if abs(v.X) > 0.00001:
        print(f'{v.VarName}: {v.X}')

Objective Value 26.0 days
assign[Terry,Customer02]: 1.0
assign[Carle,Customer03]: 1.0
assign[McClymonds,Customer01]: 1.0
